In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

ePix HR Test Structure Data Analysis

Mode TS 0

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
epixhr test structure data

:Author: Faisal Abu-Nimeh (abunimeh@slac.stanford.edu)
:License: https://confluence.slac.stanford.edu/display/ppareg/LICENSE.html
:Date: 20180629
:Style: OpenStack Style Guidelines https://docs.openstack.org/developer/hacking/
:vcs_id: $Id$
"""

# %matplotlib widget


# import h5py
import numpy as np
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
import os
import logging

# logging.basicConfig(format='%(asctime)s %(levelname)s %(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

np.set_printoptions(formatter={'int': hex})
fig_id=0

Constants

In [ ]:
#------ uint 16 ------
# Packet Size [31:16] 
# Packet Size [15:0]  
# Flags [31:16]       
# Flags [15:0]        
# x00 & Lane Number & VC Number
# x0000 
# Acquisition number [15:0]
# Acquisition number [31:16]
# x000 & 0 & ASIC number
# x0000
# Sample 0
# Sample 1
# ...
# Sample n (i.e. NSAMPLES)
#------ uint 16 ------

NSAMPLES = 30  # Number of ADC Samples in a packet
HEADER_SZ = 10  # number of 16-bit words in header
PKT_SZ = (HEADER_SZ + NSAMPLES) * 2  # Size of epixhr ts packet in bytes
PKT_WRD = NSAMPLES + HEADER_SZ  # number of words in a single packet

dac_resol = 2**16-1
dac_mem = 1024
fscale = 2.5 # 0 to 2.5 volt
tiles = 5

In [ ]:
def loadhrdata(fname):
    pBIN = np.fromfile(fname, dtype=np.uint16)
    # pBIN = np.fromfile(fname)
    filename = os.path.basename(fname)
    mdate = os.path.getmtime(fname)
    logging.debug("uint16 file size: %d" % (pBIN.size))
    
    if pBIN[0] != PKT_SZ:
        logging.error("Invalid frame size, should be: 0x%x." % (PKT_SZ))

    logging.debug("First word: 0x%x." % (pBIN[0]))
    
    step = (PKT_SZ+4)//2  # steps in 16-bit words, add additional byte by streamer
    logging.debug("step is (0x%x) %d 16-bit words" % (step,step))

    pkt_szs = pBIN[::step]
    findx = np.array_equal(pkt_szs, (np.full(pkt_szs.size, PKT_SZ, np.uint16)))
    if not findx:
        logging.error('packet sizes are not equal')
    else:
        logging.debug("Number of packets in file is %d" % (pkt_szs.shape[0]))

    if (PKT_SZ+4) * pkt_szs.shape[0] != pBIN.size*2:
        logging.error('file contains partial frames')

    # # skip headers and keep data only
    pkt_idx = step*np.arange(pkt_szs.shape[0]) + HEADER_SZ
    data_idx = np.arange(NSAMPLES) + pkt_idx[:, np.newaxis]
    data = pBIN[data_idx]
    # drop 1st 0x0 sample
    data = data[:, 1:]
    print(data)
    logging.debug("Data shape is" + str(data.shape))
    return data.copy()

Load data file

In [ ]:
# load dat file

data_ts_dac_b_ramp_125MHz = loadhrdata('datadump/20180910_122526.dat')  # ts_dac_b_ramp_125MHz_0.7scale_OSR128
data_ts_dac_b_rampf_125MHz = loadhrdata('datadump/20180910_125104.dat') # ts_dac_b_rampf_125MHz_0.7scale_OSR128
# data_ts_dac_a_ramp_125MHz = loadhrdata('datadump/20180910_105827.dat') # ts_dac_a_ramp_125MHz_0.7scale_OSR128
data_ts_dac_a_ramp_125MHz = loadhrdata('datadump/20180911_134517.dat') # ts_dac_a_ramp_125MHz_0.7scale_OSR128
# data_ts_dac_a_rampf_125MHz = loadhrdata('datadump/20180910_111954.dat') # ts_dac_a_rampf_125MHz_0.7scale_OSR128
data_ts_dac_a_rampf_125MHz = loadhrdata('datadump/20180911_140549.dat') # ts_dac_a_rampf_125MHz_0.7scale_OSR128


In [ ]:
# equalize ramps -- manually ... yeah
print(data_ts_dac_b_rampf_125MHz.shape)
print(data_ts_dac_a_rampf_125MHz.shape)

# data_ts_dac_b_ramp_125MHz_trim = data_ts_dac_b_ramp_125MHz[380:10619,:]
# data_ts_dac_a_ramp_125MHz_trim = data_ts_dac_a_ramp_125MHz[880:11119,:]

num_of_periods = 10
ramp_b_start = 380 # align sine -- manually ... yeah
ramp_a_start = 880 # align sine -- manually ... yeah
data_ts_dac_b_ramp_125MHz_trim = data_ts_dac_b_ramp_125MHz[ramp_b_start:ramp_b_start+num_of_periods*dac_mem,:]
data_ts_dac_a_ramp_125MHz_trim = data_ts_dac_a_ramp_125MHz[ramp_a_start:ramp_a_start+num_of_periods*dac_mem,:]


fig_id+=1
sample_id = 20  #  sample of interest
plt.figure(fig_id,figsize=(8,6),dpi=150)
# plt.legend(frameon=False)
plt.xlabel('Samples (Bins)')
plt.ylabel('Amplitude (ADU)')
plt.title('ePix HR Test Structure Output Rising Ramp 0.7 Scale in')
plt.plot(data_ts_dac_b_ramp_125MHz_trim, label='Diff')
plt.plot(data_ts_dac_a_ramp_125MHz_trim, label='SE')
# plt.legend(frameon=False, loc=2)
plt.show()

In [ ]:
# equalize ramps -- manually ... yeah
print(data_ts_dac_b_rampf_125MHz.shape)
print(data_ts_dac_a_rampf_125MHz.shape)

data_ts_dac_b_rampf_125MHz_trim = data_ts_dac_b_rampf_125MHz[157:11420,:]
data_ts_dac_a_rampf_125MHz_trim = data_ts_dac_a_rampf_125MHz[501:11764,:]


fig_id+=1
sample_id = 20  #  sample of interest
plt.figure(fig_id,figsize=(8,6),dpi=150)
# plt.legend(frameon=False)
plt.xlabel('Samples (Bins)')
plt.ylabel('Amplitude (ADU)')
plt.title('ePix HR Test Structure Output Rising Ramp 0.7 Scale in')
#plt.plot(data_ts_dac_b_rampf_125MHz_trim[:,sample_id], label='Diff')
#plt.plot(data_ts_dac_a_rampf_125MHz_trim[:,sample_id], label='SE')
# plt.plot(data_ts_dac_b_rampf_125MHz_trim, label='Diff')
plt.plot(data_ts_dac_a_rampf_125MHz_trim, label='SE')
# plt.legend(frameon=False, loc=2)
plt.show()

In [ ]:
# approximation of histogram
fig_id+=1
plt.figure(1,figsize=(8,6),dpi=150)
d_df_codes = data_ts_dac_b_ramp_125MHz_trim.ravel()
d_df_codes_f = data_ts_dac_b_rampf_125MHz_trim.ravel()

plt.hist(d_df_codes, bins='auto', histtype='step', label='Rising Ramp')
plt.hist(d_df_codes_f, bins='auto', histtype='step', label='Falling Ramp')

# plt.hist(d_df_codes, bins=np.arange(d_df_codes.min(),d_df_codes.max()+1), label='Rising Ramp')
# plt.hist(d_df_codes_f, bins=np.arange(d_df_codes_f.min(),d_df_codes_f.max()+1), label='Falling Ramp')


plt.xlabel('ADC Output Code')
plt.ylabel('Code Count')
plt.title('ePix HR Histogram, 125MHz, Diff, OSR128')
plt.legend(frameon=False, loc=0)
plt.show()

In [ ]:
# approximation of histogram
fig_id+=1
plt.figure(1,figsize=(8,6),dpi=150)
d_se_codes = data_ts_dac_a_ramp_125MHz_trim.ravel()
d_se_codes_f = data_ts_dac_a_rampf_125MHz_trim.ravel()

plt.hist(d_se_codes, bins='auto', histtype='step', label='Rising Ramp')
plt.hist(d_se_codes_f, bins='auto', histtype='step', label='Falling Ramp')

plt.xlabel('ADC Output Code')
plt.ylabel('Code Count')
plt.title('ePix HR Histogram, 125MHz, Single-Ended, OSR128')
plt.legend(frameon=False, loc=2)
plt.show()

In [ ]:
d_df_codes_vc = np.unique(d_df_codes, return_counts=True)  # val , cnt
d_df_codes_f_vc = np.unique(d_df_codes_f, return_counts=True) # val , cnt
d_se_codes_vc = np.unique(d_se_codes, return_counts=True) # val , cnt
d_se_codes_f_vc = np.unique(d_se_codes_f, return_counts=True) # val , cnt

d_df_bins = np.asarray((d_df_codes_vc[0], d_df_codes_vc[1])).T
d_df_f_bins = np.asarray((d_df_codes_f_vc[0], d_df_codes_f_vc[1])).T

d_se_bins = np.asarray((d_se_codes_vc[0], d_se_codes_vc[1])).T
d_se_f_bins = np.asarray((d_se_codes_f_vc[0], d_se_codes_f_vc[1])).T

In [ ]:
#d_df_bins_sorted = np.sort(d_df_bins, axis=0)
#d_df_f_bins_sorted = np.sort(d_df_f_bins, axis=0)
#d_se_bins_sorted = np.sort(d_se_bins, axis=0)
#d_se_f_bins_sorted = np.sort(d_se_f_bins, axis=0)

idx = np.argsort(d_df_bins[:,1])
d_df_bins_sorted = d_df_bins[idx]
print(np.max(d_df_bins_sorted[:,1]))

idx = np.argsort(d_df_f_bins[:,1])
d_df_f_bins_sorted = d_df_f_bins[idx]
print(np.max(d_df_f_bins_sorted[:,1]))

idx = np.argsort(d_se_bins[:,1])
d_se_bins_sorted = d_se_bins[idx]
print(np.max(d_se_bins_sorted[:,1]))

idx = np.argsort(d_se_f_bins[:,1])
d_se_f_bins_sorted = d_se_f_bins[idx]
print(np.max(d_se_f_bins_sorted[:,1]))

In [ ]:
fig_id+=1
plt.figure(fig_id,figsize=(8,6),dpi=150)
plt.xlabel('ADC Output Code')
plt.ylabel('Code Count')
plt.title('ePix HR Histogram, 125MHz, Diff, Ramp Rising')
plt.bar(d_df_bins[:,0], d_df_bins[:,1])
plt.show()

In [ ]:
fig_id+=1
plt.figure(fig_id,figsize=(8,6),dpi=150)
plt.xlabel('ADC Output Code')
plt.ylabel('Code Count')
plt.title('ePix HR Histogram, 125MHz, Diff, Ramp Rising')
plt.bar(d_df_bins_sorted[:,0], d_df_bins_sorted[:,1])
plt.show()

In [ ]:
fig_id+=1
plt.figure(fig_id,figsize=(8,6),dpi=150)
plt.xlabel('ADC Output Code')
plt.ylabel('Code Count')
plt.title('ePix HR Histogram, 125MHz, Diff, Ramp Falling')
plt.bar(d_df_f_bins[:,0], d_df_f_bins[:,1])
plt.show()

In [ ]:
fig_id+=1
plt.figure(fig_id,figsize=(8,6),dpi=150)
plt.xlabel('ADC Output Code')
plt.ylabel('Code Count')
plt.title('ePix HR Histogram, 125MHz, SE, Ramp Rising')
plt.bar(d_se_bins[:,0], d_se_bins[:,1])
plt.show()

In [ ]:
fig_id+=1
plt.figure(fig_id,figsize=(8,6),dpi=150)
plt.xlabel('ADC Output Code')
plt.ylabel('Code Count')
plt.title('ePix HR Histogram, 125MHz, SE, Ramp Falling')
plt.bar(d_se_f_bins[:,0], d_se_f_bins[:,1])
plt.show()

In [ ]:
one = data_ts_dac_a_ramp_125MHz_trim[:,0]
print(one.shape)

x = np.arange(0,10240).reshape(-1, 1024)
print(x)
print(x[0,:])
print(x.shape)

one_s = one[x[0,:]]
one_s = one[x].T
print(one_s.shape)
fig_id+=1
sample_id = 20  #  sample of interest
plt.figure(fig_id,figsize=(8,6),dpi=150)
# plt.legend(frameon=False)
plt.xlabel('Samples (Bins)')
plt.ylabel('Amplitude (ADU)')
plt.plot(one_s, label='SE')
plt.show()


In [ ]:
ramps = data_ts_dac_a_ramp_125MHz_trim
print(ramps.shape)
x = ramps.shape[0] / num_of_periods
y = ramps.shape[1] * num_of_periods
single_ramp = ramps.reshape(int(x),int(y), order='F')
print(single_ramp.shape)
print(single_ramp)

# trim
# remove 12 from start and 12 from end
single_ramp = single_ramp[11:1011, :]
print(single_ramp.shape)

fig_id+=1
plt.figure(fig_id,figsize=(8,6),dpi=150)

xl = np.arange(single_ramp.shape[0])
xlflat = np.tile(xl, single_ramp.shape[1])
ylflat = single_ramp.T.ravel()
slope, intercept, r_value, p_value, std_err = stats.linregress(xlflat, ylflat)
plt.plot(single_ramp, 'k.')
plt.plot(xl, intercept+slope*xl, '-', linewidth=2, label='linear fit: ' +r'$%5.2fx+%5.2f$' % (slope, intercept))
plt.legend(frameon=False)
plt.show()

print('slope %.3f' % (slope))
print('intercept %.3f' % (intercept))
print('R2 %.3f' % (r_value**2))
print('p %.3f' % (p_value))
print('std_err %.3f' % (std_err))

In [ ]:
# FIXME use correct axis
# plt.figure(1,figsize=(8,6),dpi=150)
# ADCRESOL = 16  # 16-bit ADC
# ramp_avg = np.average(single_ramp.T, 0)
# hcoeff = np.polyfit(xl, ramp_avg, 1)  # first order polyfit for tail
# hfit1 = np.poly1d(hcoeff)
# hyfit = hfit1(xl)
# plt.plot(xl, 100 * (hyfit - ramp_avg) / (2**ADCRESOL), '.')
# plt.title('INL')
# plt.xlabel('ADC Code')
# plt.ylabel('INL [% Full Scale Range]')
# fig = plt.gcf()
# plt.show()